In [1]:
from skimage.feature import hog
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import glob
from sklearn.decomposition import PCA

In [2]:
images_path = '/Users/joanna/datasci_281/final_project/HAM10000_images_preprocessed'
segmentation_path = '/Users/joanna/datasci_281/final_project/HAM10000_segmentations_lesion_tschandl'
metadata = pd.read_csv('HAM10000_metadata.csv')

# get paths for image and segmentation files
img_files = sorted(glob.glob('/'.join([images_path, '*.jpg'])))
segmentation_files = sorted(glob.glob('/'.join([segmentation_path, '*.png'])))
# get the image ID 
img_files_df = pd.DataFrame({'img_file_path': img_files})
img_files_df['image_id'] = img_files_df['img_file_path'].apply(lambda x: x.split('/')[-1].replace('.jpg', ''))
# get the segmentation ID
segmentation_files_df = pd.DataFrame({'segmentation_file_path': segmentation_files})
segmentation_files_df['image_id'] = segmentation_files_df['segmentation_file_path'].apply(lambda x: x.split('/')[-1].replace('_segmentation.png', ''))

# merge the dataframes to make one metadata file
metadata = pd.merge(metadata, img_files_df, left_on= 'image_id', right_on = 'image_id')
metadata = pd.merge(metadata, segmentation_files_df, left_on = 'image_id', right_on = 'image_id')

metadata.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,img_file_path,segmentation_file_path
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,/Users/joanna/datasci_281/final_project/HAM100...,/Users/joanna/datasci_281/final_project/HAM100...
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,/Users/joanna/datasci_281/final_project/HAM100...,/Users/joanna/datasci_281/final_project/HAM100...
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,/Users/joanna/datasci_281/final_project/HAM100...,/Users/joanna/datasci_281/final_project/HAM100...
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,/Users/joanna/datasci_281/final_project/HAM100...,/Users/joanna/datasci_281/final_project/HAM100...
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,/Users/joanna/datasci_281/final_project/HAM100...,/Users/joanna/datasci_281/final_project/HAM100...


In [38]:
img_info = metadata[['image_id','img_file_path', 'segmentation_file_path']]
feature_df = pd.DataFrame()
for i in range(len(metadata)):
    segmentation = cv2.imread(img_info['segmentation_file_path'].iloc[i], cv2.IMREAD_GRAYSCALE)
    hog_features = hog(segmentation)
    feature = pd.DataFrame({'image_id': [img_info['image_id'].iloc[i]],
                           'hog_average': [hog_features]})
    feature_df = pd.concat([feature_df, feature], ignore_index= True)

In [40]:
feature_df.head()

,image_id,hog_average
0,ISIC_0027419,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,ISIC_0025030,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,ISIC_0026769,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,ISIC_0025661,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,ISIC_0031633,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [69]:
metadata

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,img_file_path,segmentation_file_path
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,/Users/joanna/datasci_281/final_project/HAM100...,/Users/joanna/datasci_281/final_project/HAM100...
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,/Users/joanna/datasci_281/final_project/HAM100...,/Users/joanna/datasci_281/final_project/HAM100...
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,/Users/joanna/datasci_281/final_project/HAM100...,/Users/joanna/datasci_281/final_project/HAM100...
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,/Users/joanna/datasci_281/final_project/HAM100...,/Users/joanna/datasci_281/final_project/HAM100...
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,/Users/joanna/datasci_281/final_project/HAM100...,/Users/joanna/datasci_281/final_project/HAM100...
...,...,...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,vidir_modern,/Users/joanna/datasci_281/final_project/HAM100...,/Users/joanna/datasci_281/final_project/HAM100...
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,vidir_modern,/Users/joanna/datasci_281/final_project/HAM100...,/Users/joanna/datasci_281/final_project/HAM100...
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,vidir_modern,/Users/joanna/datasci_281/final_project/HAM100...,/Users/joanna/datasci_281/final_project/HAM100...
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,vidir_modern,/Users/joanna/datasci_281/final_project/HAM100...,/Users/joanna/datasci_281/final_project/HAM100...


In [45]:
from sklearn.decomposition import PCA

hog_matrix = feature_df['hog_average'].tolist()
hog_matrix = np.array(hog_matrix)

pca = PCA(n_components = 10)
value_features = pca.fit_transform(hog_matrix)

In [71]:
feature_df = pd.DataFrame({'image_id': metadata['image_id']})

In [72]:
for i in range(10):
    feature = pd.DataFrame({'_'.join(['hog_feature', str(i)]): value_features[:,i]})
    feature_df = pd.concat([feature_df, feature], axis = 1)

In [74]:
feature_df.to_csv('hog_features.csv', index = False)